In [13]:
import sys
sys.path.append('../..')

import telnyx
import json

from src import getSecret
import base64
# class Telnyx_API:
    # def __init__(self, db, csrf):
telnyx.api_key = getSecret('TELNYX_API_KEY')
telnyx.public_key = getSecret('TELNYX_PUBLIC_KEY')
# self.db = db
# self.csrf = csrf
public_key = getSecret('TELNYX_PUBLIC_KEY')
# base64
# public_key =

tolerance = 300 # 5 minutes
tolerance = '600'

payload = "{\n  \"data\": {\n    \"event_type\": \"message.received\",\n    \"id\": \"e51572e0-c4dc-413f-90e8-541926a78fb8\",\n    \"occurred_at\": \"2023-04-25T18:30:11.451+00:00\",\n    \"payload\": {\n      \"autoresponse_type\": null,\n      \"cc\": [],\n      \"completed_at\": null,\n      \"cost\": null,\n      \"direction\": \"inbound\",\n      \"encoding\": \"GSM-7\",\n      \"errors\": [],\n      \"from\": {\n        \"carrier\": \"Verizon Wireless\",\n        \"line_type\": \"Wireless\",\n        \"phone_number\": \"+17743642453\"\n      },\n      \"id\": \"f4f13a15-3cbc-4d01-a91c-86008bca2484\",\n      \"media\": [],\n      \"messaging_profile_id\": \"40017f0f-92d8-4318-9f8a-e0e2a92c3f60\",\n      \"organization_id\": \"82fc37e7-63d3-4165-9b54-855ee8caa737\",\n      \"parts\": 1,\n      \"received_at\": \"2023-04-25T18:30:11.351+00:00\",\n      \"record_type\": \"message\",\n      \"sent_at\": null,\n      \"subject\": \"\",\n      \"tags\": [],\n      \"text\": \"14:30\",\n      \"to\": [\n        {\n          \"carrier\": \"Telnyx\",\n          \"line_type\": \"Wireless\",\n          \"phone_number\": \"+16182721034\",\n          \"status\": \"webhook_delivered\"\n        }\n      ],\n      \"type\": \"SMS\",\n      \"valid_until\": null,\n      \"webhook_failover_url\": null,\n      \"webhook_url\": \"https://pulse-fi.herokuapp.com/forwardSMS\"\n    },\n    \"record_type\": \"event\"\n  },\n  \"meta\": {\n    \"attempt\": 1,\n    \"delivered_to\": \"https://pulse-fi.herokuapp.com/forwardSMS\"\n  }\n}"
# convert to bytes
payload = payload.encode('utf-8')
signature = '0oWL5HazU5ylxuLoQTeNC2y7ea3BJ2TFIWzMcMjGrL00jhaRe7kbF7BUZPynHqtv4pIckTIj/o2du10vW23dAQ=='
timestamp = '1682447412'

# event = telnyx.Webhook.construct_event(payload, signature, timestamp, public_key, tolerance)


from nacl.encoding import Base64Encoder
from nacl.signing import VerifyKey
        
from telnyx import error
import time


# class WebhookSignature(object):
    # @classmethod
def verify(payload, signature, timestamp, tolerance=None):
    if hasattr(timestamp, "encode"):
        timestamp = timestamp.encode("utf-8")

    if hasattr(payload, "encode"):
        payload = payload.encode("utf-8")

    signed_payload = timestamp + b"|" + payload
    public_key = telnyx.public_key

    if not public_key:
        raise error.SignatureVerificationError(
            "Public key not set", signature, payload
        )

    # verify the data
    key = VerifyKey(public_key, encoder=Base64Encoder)

    if not key.verify(signed_payload, signature=base64.b64decode(signature)):
        raise error.SignatureVerificationError(
            "Signature is invalid and does not match the payload",
            signature,
            payload,
        )

    if tolerance and int(timestamp) < time.time() - tolerance:
        raise error.SignatureVerificationError(
            "Timestamp outside the tolerance zone (%s)" % timestamp,
            signature,
            payload,
        )

    return True

DEFAULT_TOLERANCE = 300
def construct_event(
    payload, signature_header, timestamp, tolerance=DEFAULT_TOLERANCE, api_key=None
):

    if verify(payload, signature_header, timestamp, tolerance):

        if api_key is None:
            api_key = telnyx.api_key

        if hasattr(payload, "decode"):
            payload = payload.decode("utf-8")

        data = json.loads(payload)
        event = telnyx.Event.construct_from(data, api_key)

        return event
    
event = construct_event(payload, signature, timestamp, tolerance)

In [2]:
from datetime import datetime
dateInt = 1682445429
# convert
date = datetime.fromtimestamp(dateInt).strftime('%Y-%m-%d %H:%M:%S')
Default_tolerance = 300
dateTolerance = datetime.fromtimestamp(dateInt + Default_tolerance).strftime('%Y-%m-%d %H:%M:%S')
print(date)
print(dateTolerance)

2023-04-25 13:57:09
2023-04-25 14:02:09


In [ ]:
import requests

url = "https://pulse-fi.herokuapp.com/forwardSMS"

# Define your dummy data as a dictionary
dummy_data = {
    "key1": "value1",
    "key2": "value2",
    "key3": "value3"
}

# Send the POST request with the dummy data
response = requests.post(url, data=dummy_data)

# Print the response status code and content
print("Status code:", response.status_code)
print("Response content:", response.text)

In [ ]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import requests

import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager

# Replace with your Google email and password
googleUsername = getSecret('GOOGLE_USERNAME')
googlePassword = getSecret('GOOGLE_PASSWORD')

url = 'https://myactivity.google.com/product/youtube'

# Set up the web driver (replace with the path to your driver if needed)
driver = webdriver.Chrome(ChromeDriverManager().install())

# Navigate to the URL
driver.get(url)

# Log in to your Google account
email_input = driver.find_element("xpath",'//*[@id="identifierId"]')
email_input.send_keys(googleUsername)
email_input.send_keys(Keys.RETURN)
time.sleep(2)  # Wait for the password field to appear

password_input = driver.find_element("xpath",'//*[@id="password"]/div[1]/div/div[1]/input')
password_input.send_keys(googlePassword)
password_input.send_keys(Keys.RETURN)
time.sleep(5)  # Wait for the page to load

# Get the page's HTML content
html_content = driver.page_source

# Close the browser window
driver.quit()

# Process the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')
print(soup.prettify())


# html_content = '''
# PASTE YOUR HTML CONTENT HERE
# '''

# soup = BeautifulSoup(html_content, 'html.parser')

# videos = soup.find_all('div', class_='uUy2re')

# for video in videos:
#     title_element = video.find('a', class_='l8sGWb')
#     channel_element = video.find('a', target='_blank', rel='noopener noreferrer')
    
#     title = title_element.text
#     channel = channel_element.text
#     url = title_element['href']

#     print(f"Title: {title}\nChannel: {channel}\nURL: {url}\n")

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()

email = getSecret('GOOGLE_USERNAME')
password = getSecret('GOOGLE_PASSWORD')

def Google_Sign_in():

    driver.get("https://accounts.google.com/signin/v2/identifier?flowName=GlifWebSignIn&flowEntry=ServiceLogin")
    
    ## Puts in email
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "identifierId"))).send_keys(email)

    driver.find_element("id","identifierNext").click()

    ##Puts in password
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.NAME, "password"))).send_keys(password)

    driver.find_element("id","passwordNext").click()

Google_Sign_in()

In [ ]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import undetected_chromedriver as uc
import random,time,os,sys
from selenium.webdriver.common.keys import Keys

GMAIL = getSecret('GOOGLE_USERNAME')
PASSWORD = getSecret('GOOGLE_PASSWORD')

chrome_options = uc.ChromeOptions()
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--profile-directory=Default")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--disable-plugins-discovery")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("user_agent=DN")
chrome_options.add_argument("--disable-web-security")
chrome_options.add_argument("--user-data-dir")
chrome_options.add_argument("--allow-running-insecure-content")

driver = uc.Chrome(options=chrome_options)

driver.delete_all_cookies()

driver.get("https://accounts.google.com/o/oauth2/v2/auth/oauthchooseaccount?redirect_uri=https%3A%2F%2Fdevelopers.google.com%2Foauthplayground&prompt=consent&response_type=code&client_id=407408718192.apps.googleusercontent.com&scope=email&access_type=offline&flowName=GeneralOAuthFlow")

driver.find_element("xpath","/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div/div[1]/div/div[1]/input").send_keys(GMAIL)
driver.find_element("xpath","/html/body/div[1]/div[1]/div[2]/div/div[2]/div/div/div[2]/div/div[1]/div/form/span/section/div/div/div[1]/div/div[1]/div/div[1]/input").send_keys(Keys.RETURN)

time.sleep(10)

In [ ]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import undetected_chromedriver as uc
import random,time,os,sys
from selenium.webdriver.common.keys import Keys

GMAIL = getSecret('GOOGLE_USERNAME')
PASSWORD = getSecret('GOOGLE_PASSWORD')

chrome_options = uc.ChromeOptions()
# chrome_options.add_argument("--disable-extensions")
# chrome_options.add_argument("--disable-popup-blocking")
# chrome_options.add_argument("--profile-directory=Default")
# chrome_options.add_argument("--ignore-certificate-errors")
# chrome_options.add_argument("--disable-plugins-discovery")
# chrome_options.add_argument("--incognito")
# chrome_options.add_argument("user_agent=DN")
# chrome_options.add_argument("--disable-web-security")
# chrome_options.add_argument("--user-data-dir")
# chrome_options.add_argument("--allow-running-insecure-content")

driver = uc.Chrome(options=chrome_options)

driver.delete_all_cookies()

driver.get("https://stackoverflow.com/users/login")

google_login_button = driver.find_element('xpath','//button[@data-provider="google"]')
google_login_button.click()

time.sleep(2)

email_field = driver.find_element('xpath','//*[@id="identifierId"]')
email_field.send_keys(GMAIL)
email_field.send_keys(Keys.ENTER)

time.sleep(2)

password_field = driver.find_element('xpath','//*[@id="password"]/div[1]/div/div[1]/input')
password_field.send_keys(PASSWORD)
password_field.send_keys(Keys.ENTER)

time.sleep(5)

# wait for the 2 factor authenication 


driver.get("https://myactivity.google.com/product/youtube")

# The browser should now be logged in to Google and displaying the desired URL
# To close the browser, uncomment the following line
# driver.quit()


In [ ]:
getSecret('GOOGLE_USERNAME')
getSecret('GOOGLE_PASSWORD')

In [ ]:
import sys
import os
sys.path.append('../..')
from src import getSecret

import undetected_chromedriver as uc
import random,time,os,sys
from selenium.webdriver.common.keys import Keys

GMAIL = getSecret('GOOGLE_USERNAME')
PASSWORD = getSecret('GOOGLE_PASSWORD')

# Use the appropriate WebDriver for your browser
driver = webdriver.Chrome()
# driver = webdriver.Firefox()

# Navigate to the Stack Overflow login page
driver.get("https://stackoverflow.com/users/login")

# Click the "Log in with Google" button
google_login_button = driver.find_element("xpath",'//button[@data-provider="google"]')
google_login_button.click()

# Wait for the email field to appear
time.sleep(2)

# Fill in your Google email and press Enter
email_field = driver.find_element("xpath",'//*[@id="identifierId"]')
email_field.send_keys(GMAIL)
email_field.send_keys(Keys.ENTER)

# Wait for the password field to appear
time.sleep(2)

# Fill in your Google password and press Enter
password_field = driver.find_element("xpath",'//*[@id="password"]/div[1]/div/div[1]/input')
password_field.send_keys(PASSWORD)
password_field.send_keys(Keys.ENTER)

# Wait for the page to load
time.sleep(5)

# Navigate to the Google URL
driver.get("https://myactivity.google.com/product/youtube")

# The browser should now be logged in to Google and displaying the desired URL
# To close the browser, uncomment the following line
# driver.quit()


In [ ]:
import sys
import os
sys.path.append('../..')

import google.auth
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set up API client
def get_authenticated_service():
    # Replace 'client_secrets.json' with the name of your downloaded JSON client secrets file
    flow = InstalledAppFlow.from_client_secrets_file('../../secrets/oauth2.0_youtubeAPI_secretKey.json', ['https://www.googleapis.com/auth/youtube.readonly'])
    credentials = flow.run_local_server()
    return build('youtube', 'v3', credentials=credentials)

def get_watch_history(youtube):
    try:
        results = youtube.playlistItems().list(
            part="snippet",
            playlistId="HL",  # HL is the playlist ID for watch history
            maxResults=50  # You can set this value up to 50
        ).execute()

        for item in results["items"]:
            title = item["snippet"]["title"]
            video_id = item["snippet"]["resourceId"]["videoId"]
            print(f"{title}: https://www.youtube.com/watch?v={video_id}")

    except HttpError as error:
        print(f"An error occurred: {error}")
        return None

# if __name__ == "__main__":
youtube_service = get_authenticated_service()
get_watch_history(youtube_service)
